// 原始语料在 doc\lang\programming\pytorch\机器翻译\中英平行语料520万_translation2019zh  格式是：每行一个JSON 文本， 有两个字段 english 和 chinese 
// 转换为 OpenNMT-py 格式，英文一个文件，中文一个文件

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')
! gdown --id '1vzW8NXWmJ9LHp9hYs4u6SsgWtTF4sd7z'
! unzip ./en_chs.zip
#! unzip ./chinese_roberta_wwm_ext_L-12_H-768_A-12.zip -d ./chinese_roberta_wwm_ext_L-12_H-768_A-12


In [ ]:
!nvidia-smi

In [ ]:
!pip install --upgrade pip
!pip install OpenNMT-py

In [ ]:
%%bash
cat <<EOF > en_chs.yaml
# toy_en_de.yaml

## Where the samples will be written
save_data: en_chs/run/example
## Where the vocab(s) will be written
src_vocab: en_chs/run/example.vocab.src
tgt_vocab: en_chs/run/example.vocab.tgt
# Prevent overwriting existing files in the folder
overwrite: True

# Corpus opts:
data:
    corpus_1:
        path_src: en_chs/src-train.txt
        path_tgt: en_chs/tgt-train.txt
    valid:
        path_src: en_chs/src-val.txt
        path_tgt: en_chs/tgt-val.txt

EOF

From this configuration, we can build the vocab(s), that will be necessary to train the model:

In [ ]:
!onmt_build_vocab -config en_chs.yaml -n_sample 20000
#!onmt_build_vocab -config en_chs.yaml -n_sample 10000

Notes:

-n_sample is required here – it represents the number of lines sampled from each corpus to build the vocab.

This configuration is the simplest possible, without any tokenization or other transforms. See other example configurations for more complex pipelines.

**Step 2: Train the model**

To train a model, we need to add the following to the YAML configuration file:

the vocabulary path(s) that will be used: can be that generated by onmt_build_vocab;

training specific parameters.

In [ ]:
%%bash 
cat <<EOF >> en_chs.yaml
# Vocabulary files that were just created
src_vocab: en_chs/run/example.vocab.src
tgt_vocab: en_chs/run/example.vocab.tgt

# Train on a single GPU
world_size: 1
gpu_ranks: [0]

# Where to save the checkpoints
save_model: en_chs/run/model
save_checkpoint_steps: 10000
train_steps: 10000
valid_steps: 5000
EOF

Then you can simply run:

In [ ]:
!onmt_train -config en_chs.yaml

This configuration will run the default model, which consists of a 2-layer LSTM with 500 hidden units on both the encoder and decoder. It will run on a single GPU (world_size 1 & gpu_ranks [0]).

Before the training process actually starts, the *.vocab.pt together with *.transforms.pt can be dumped to -save_data with configurations specified in -config yaml file by enabling the -dump_fields and -dump_transforms flags. It is also possible to generate transformed samples to simplify any potentially required visual inspection. The number of sample lines to dump per corpus is set with the -n_sample flag.

For more advanded models and parameters, see other example configurations or the FAQ.

**Step 3: Translate**

In [ ]:
!onmt_translate -model en_chs/run/model_step_10000.pt -src en_chs/src-test.txt -output en_chs/pred_10000.txt -gpu 0 -verbose

Now you have a model which you can use to predict on new data. We do this by running beam search. This will output predictions into toy-ende/pred_1000.txt.

Note:

The predictions are going to be quite terrible, as the demo dataset is small. Try running on some larger datasets! For example you can download millions of parallel sentences for translation or summarization.